# Opensloth

### Installation

In [1]:
%%capture
!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124
!pip install accelerate==1.7.0
!pip install triton==3.2.0 

In [2]:
%%capture
!pip install unsloth==2025.5.7 unsloth-zoo==2025.5.8 --no-cache

In [3]:
%%capture
!pip install opensloth==0.1.7 

### Cache dataset to disk

In [4]:
%%writefile cache_unsloth_dataset.py
"""
COPY FROM: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen3_(14B)-Reasoning-Conversational.ipynb#scrollTo=5kyTw2n1edte
"""

from typing import Any

import pandas as pd
from datasets import Dataset, load_dataset
from trl import SFTConfig, SFTTrainer
from unsloth import FastLanguageModel


def dump_data() -> Any:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/qwen3-0.6b-bnb-4bit",  # Just for storing dataset, use smallest of its varient
        max_seq_length=2048,  # Context length - can be longer, but uses more memory
        load_in_4bit=True,  # 4bit uses much less memory
        load_in_8bit=False,  # A bit more accurate, uses 2x memory
        full_finetuning=False,  # We have full finetuning now!
    )

    model = FastLanguageModel.get_peft_model(
        model,
        r=32,  # Choose any number > 0! Suggested 8, 16, 32, 64, 128
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],
        lora_alpha=32,  # Best to choose alpha = rank or rank*2
        lora_dropout=0,  # Supports any, but = 0 is optimized
        bias="none",  # Supports any, but = "none" is optimized
        # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
        use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
        random_state=3407,
        use_rslora=False,  # We support rank stabilized LoRA
        loftq_config=None,  # And LoftQ
    )

    dataset = load_dataset("unsloth/OpenMathReasoning-mini", split="cot")

    def to_conv(x):
        return [
            {"role": "user", "content": x["problem"]},
            {"role": "assistant", "content": x["generated_solution"]},
        ]

    df = pd.DataFrame(dataset).sample(1000, random_state=3407)
    df["messages"] = df.apply(to_conv, axis=1)
    df["text"] = df["messages"].apply(
        lambda m: tokenizer.apply_chat_template(m, tokenize=False)
    )
    ds = Dataset.from_pandas(df[["text"]])
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=ds,
        args=SFTConfig(
            dataset_text_field="text",
            per_device_train_batch_size=2,
            gradient_accumulation_steps=4,  # Use GA to mimic batch size!
            warmup_steps=5,
            # num_train_epochs = 1, # Set this for 1 full training run.
            max_steps=30,
            learning_rate=2e-4,  # Reduce to 2e-5 for long training runs
            logging_steps=1,
            optim="adamw_8bit",
            weight_decay=0.01,
            lr_scheduler_type="linear",
            seed=3407,
            report_to="none",  # Use this for WandB etc
        ),
    )
    trainer.train_dataset.save_to_disk("data/cache_qwen3_dataset")
    print("Dataset saved to data/cache_qwen3_dataset")


if __name__ == "__main__":
    dump_data()


Writing cache_unsloth_dataset.py


In [5]:
!python cache_unsloth_dataset.py


2025-06-21 13:06:05.497468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750511165.724101     101 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750511165.787091     101 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/kaggle/working/cache_unsloth_dataset.py:10: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel
🦥 Unsloth: Will patch your computer to enable 2x faster free finetunin

## Train

In [6]:
%%writefile train.py
from opensloth.opensloth_config import (
    FastModelArgs,
    LoraArgs,
    OpenSlothConfig,
    TrainingArguments,
)
from opensloth.scripts.opensloth_sft_trainer import run_mp_training, setup_envs

# 2 GPUs with packing configuration
GLOBAL_BZ = 32

DEVICES = [0,1]

BZ = 1  # if sequence packing, then should be 1, larger does not contribute to speed
opensloth_config = OpenSlothConfig(
    data_cache_path="data/cache_qwen3_dataset/",
    devices=DEVICES,
    fast_model_args=FastModelArgs(
        model_name="unsloth/Qwen3-0.6B-Base-bnb-4bit",
        max_seq_length=16000,
        load_in_4bit=True,
    ),
    lora_args=LoraArgs(
        r=8,
        lora_alpha=16,
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],
        lora_dropout=0,
        bias="none",
        use_rslora=False,
    ),
    sequence_packing=True,
)

training_config = TrainingArguments(
    output_dir="outputs/exps/qwen3-0.6b-FineTome-2gpu-packing",
    per_device_train_batch_size=BZ,
    gradient_accumulation_steps=GLOBAL_BZ // (len(DEVICES) * BZ),
    learning_rate=1e-5,
    logging_steps=1,
    num_train_epochs=1,
    lr_scheduler_type="linear",
    warmup_steps=5,
    save_total_limit=1,
    weight_decay=0.01,
    optim="adamw_8bit",
    seed=3407,
    report_to="none",  # or wandb/tensorboard
)


if __name__ == "__main__":
    import os

    # Setup wandb
    os.environ["WANDB_PROJECT"] = "opensloth"
    os.environ["WANDB_NAME"] = (
        f"qwen3-0.6b_2gpu_packing_globalbz{GLOBAL_BZ}_samples10000"
    )

    print(
        f"Global batch size: {len(DEVICES) * BZ * training_config.gradient_accumulation_steps}"
    )
    print(f"Gradient accumulation steps: {training_config.gradient_accumulation_steps}")

    setup_envs(opensloth_config, training_config)
    run_mp_training(opensloth_config.devices, opensloth_config, training_config)


Writing train.py


In [7]:
!python train.py

Global batch size: 32
Gradient accumulation steps: 16
Global batch size: 32
[MP] Running on 2 GPUs
2025-06-21 13:07:22.458108: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-21 13:07:22.458108: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750511242.481099     288 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750511242.481098     289 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750511242.488972     288 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plug